# Pré-traitement du jeu de données

## Installation des dépendances

In [13]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


## Importation des dépendances

In [14]:
import os
from PIL import Image, ImageOps
import shutil

## Code

### Fonctions principales

In [15]:
def create_output_repository(output_path: str) -> None:
    """
    Permet de recréer le répertoire au propre.
    
    :param output_path: Chemin du répertoire.
    """
    
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    
    os.makedirs(output_path)
    os.makedirs(f"{output_path}/train")
    os.makedirs(f"{output_path}/val")
    os.makedirs(f"{output_path}/test")

In [16]:
def convert_to_png(image_path: str) -> str:
    """
    Converti une image en PNG.
    
    :param image_path: Chemin de l'image à convertire.
    """
    
    try:
        with Image.open(image_path) as img:
            # Générer le chemin de sortie avec la même base mais l'extension .png
            base_name, _ = os.path.splitext(image_path)
            output_path = base_name + ".png"
            
            # Sauvegarder l'image au format PNG
            img.save(output_path, "PNG")
            
            return output_path
    except Exception as e:
        print(f"Erreur lors de la conversion de {image_path} : {e}")

In [17]:
def resize_with_padding(image_path: str, output_path: str, size: int = 640) -> None:
    """
    Redimensionne une image tout en conservant ses proportions et ajoute du padding pour obtenir un carré de dimensions size x size.
    
    :param image_path: Chemin de l'image d'entrée.
    :param output_path: Chemin où sauvegarder l'image transformée.
    :param size: Taille du carré cible (par défaut 640).
    """
    
    image_path = convert_to_png(image_path)
    
    with Image.open(image_path) as img:
        # Convertir en mode RGB si nécessaire (prévention des problèmes avec les images en niveaux de gris ou alpha)
        img = img.convert("RGB")

        # Récupérer les dimensions de l'image
        original_width, original_height = img.size

        # Calculer le ratio d'échelle tout en conservant les proportions
        scale = min(size / original_width, size / original_height)
        new_width = int(original_width * scale)
        new_height = int(original_height * scale)

        # Redimensionner l'image tout en conservant les proportions
        img_resized = img.resize((new_width, new_height), Image.LANCZOS)

        # Ajouter du padding pour atteindre la taille cible
        padding_left = (size - new_width) // 2
        padding_top = (size - new_height) // 2
        padding_right = size - new_width - padding_left
        padding_bottom = size - new_height - padding_top

        # Ajouter des bords noirs (padding)
        img_padded = ImageOps.expand(img_resized, 
                                     border=(padding_left, padding_top, padding_right, padding_bottom), 
                                     fill=(0, 0, 0))

        # Sauvegarder l'image résultante
        img_padded.save(output_path)

In [18]:
def processing_folder(input_folder: str, output_folder: str) -> None:
    """
    Boucle sur un répertoire pour déplacer / redimensionner au bon endroit les images.
    
    :param input_folder: Chemin du répertoire d'entrée.
    :param ouput_folder: Chemin du répertoire de sortie.
    """

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)

        if os.path.isdir(input_path):
            output_path = os.path.join(output_folder, filename)
            
            processing_folder(input_path, output_path)
        else:
            folder = os.path.basename(output_folder)
            base_name, _ = os.path.splitext(filename)
            new_filename = f"{folder}_{base_name}.png"
            dirname = os.path.dirname(output_folder)
            output_path = os.path.join(dirname, new_filename)
            
            resize_with_padding(input_path, output_path)

<span style="color:red">Il faut absolument automatiser cela !!! (ça n'existe pas, il faut créer une solution)</span>

In [19]:
def get_annotation(image_path: str) -> list:
    """
    Récupère les annotations codées en dur pour une image donnée.
    
    :param image_path: Chemin de l'image.
    :return: Liste des boîtes englobantes sous forme [(xmin, ymin, xmax, ymax)].
    """
    
    annotations = {} # TODO remplir automatique
    
    image_name = image_path.split('/')[-1]
    
    return annotations.get(image_name, [[0, 0, 640, 640]])

In [20]:
def add_labels(input_folder: str, classes_dict: dict) -> None:
    """
    Crée les fichiers .txt pour chaque image dans le répertoire de sortie pour l'entraînement YOLO.
    
    :param input_folder: Chemin du répertoire contenant les images.
    :param classes_dict: Dictionnaire des classes avec leurs noms.
    :param output_folder: Chemin du répertoire où les fichiers .txt seront sauvegardés.
    """
    
    output_folder = input_folder.replace("images", "labels")
    
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)

        if os.path.isdir(input_path):            
            add_labels(input_path, classes_dict)
        else:
            class_name = filename.split('_')[0]
            
            if class_name not in classes_dict:
                class_id = classes_dict["notfound"]
            else:
                class_id = classes_dict[class_name]
            
            label_file = os.path.join(output_folder, f"{filename.split('.')[0]}.txt")
            
            annotations = get_annotation(filename)
            
            with open(label_file, 'w') as f:
                for annotation in annotations:
                    # Normaliser les coordonnées
                    xmin, ymin, xmax, ymax = annotation
                        
                    # Obtenir la taille de l'image
                    with Image.open(input_path) as img:
                        width, height = img.size
                            
                    # Calculer les coordonnées normalisées (entre 0 et 1)
                    x_center = (xmin + xmax) / 2 / width
                    y_center = (ymin + ymax) / 2 / height
                    obj_width = (xmax - xmin) / width
                    obj_height = (ymax - ymin) / height
                        
                    # Écrire l'annotation dans le fichier
                    f.write(f"{class_id} {x_center} {y_center} {obj_width} {obj_height}\n")


### Environnement

In [21]:
input_folder = "./dataset1/images_base"
output_folder = "./dataset1/images_converted"
classes_dict = { "SpeedLimit30": 0, "SpeedLimit50": 1, "SpeedLimit70": 2, "SpeedLimit80": 3, "SpeedLimit90": 4, "SpeedLimit110": 5, "SpeedLimit130": 6 }

### Lancement

In [22]:
create_output_repository(output_folder)

In [23]:
processing_folder(input_folder, output_folder)

In [24]:
# add_labels(output_folder, classes_dict)